In [1]:
from torch.distributions import Binomial

import torch
import numpy as np

/home/cbelem/miniconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def tensor2numpy(t):
    if isinstance(t, torch.Tensor):
        return t.cpu().numpy()
    else:
        return t


def quantile_intervals(
    n,  # (int) Number of data points in your original sample 
    desired_quantiles,  # (1D torch.tensor, dtype=float) Contains values between 0 and 1 indicating the quantiles you want intervals for
    desired_confidence=0.95,  # (float) Single confidence level you want the intervals to span
):
    desired_ranks = desired_quantiles.double()*n
    possible_ranks = torch.arange(0, n, 1).double()
    dists = Binomial(total_count=n*torch.ones_like(desired_ranks), probs=desired_quantiles.double())

    pmfs = dists.log_prob(possible_ranks.unsqueeze(1)).exp().T
    cdfs = pmfs.cumsum(dim=1)

    dists = cdfs.unsqueeze(-1) - cdfs.unsqueeze(-2)  # (batch, upper_interval_idx, lower_interval_idx)
    valid_upper_indices = possible_ranks.unsqueeze(0) > desired_ranks.unsqueeze(-1)
    valid_lower_indices = possible_ranks.unsqueeze(0) < desired_ranks.unsqueeze(-1)
    valid_indices = valid_upper_indices.unsqueeze(-1) & valid_lower_indices.unsqueeze(-2)

    valid_confs = dists >= desired_confidence
    valid_dists = torch.where(valid_indices & valid_confs, dists, torch.finfo(torch.float64).max)

    interval_info = valid_dists.view(valid_dists.shape[0], -1).min(dim=-1)
    interval_indices, interval_widths = interval_info.indices, interval_info.values
    interval_indices = torch.stack([interval_indices // n, interval_indices % n], -1)

    upper_interval_ranks = interval_indices[:, 0]
    upper_interval_quantiles = upper_interval_ranks / n
    lower_interval_ranks = interval_indices[:, 1]
    lower_interval_quantiles = lower_interval_ranks / n

    metadata = {
        "n": n,
        "desired_confidence": desired_confidence,
        "desired_quantiles": desired_quantiles,
        "desired_ranks": desired_ranks,
        # For whatever sample of results you have that you are taking quantiles on, the interval will be defined be other specific quantiles of the same sample
        "upper_interval_quantiles": upper_interval_quantiles,
        "lower_interval_quantiles": lower_interval_quantiles,
        # These are the exact ranks of the interval endpoints. i.e., if the upper_interval_rank=4, then the upper end of the interval is the 4th largest value in the sample 
        "upper_interval_ranks": upper_interval_ranks,
        "lower_interval_ranks": lower_interval_ranks,
        # The estimated intervals are not exact, and will often span more than the desired confidence. A good sanity check is to make sure the interval widths are close enough.
        "interval_widths": interval_widths,
    }
    
    return {k: tensor2numpy(v) for k, v in metadata.items()}

In [4]:
n = 15000
desired_quantiles = torch.tensor([0.1, 0.5, 0.75, 0.9, 0.99, 0.999], dtype=torch.float64)
desired_confidence = 0.95
quantile_intervals(n, desired_quantiles, desired_confidence)

{'n': 15000,
 'desired_confidence': 0.95,
 'desired_quantiles': array([0.1  , 0.5  , 0.75 , 0.9  , 0.99 , 0.999]),
 'desired_ranks': array([ 1500.,  7500., 11250., 13500., 14850., 14985.]),
 'upper_interval_quantiles': array([0.10406667, 0.50673336, 0.7622667 , 0.90686667, 0.99233335,
        0.9994    ], dtype=float32),
 'lower_interval_quantiles': array([0.09306667, 0.4876    , 0.74413335, 0.8958667 , 0.9886    ,
        0.99833333], dtype=float32),
 'upper_interval_ranks': array([ 1561,  7601, 11434, 13603, 14885, 14991]),
 'lower_interval_ranks': array([ 1396,  7314, 11162, 13438, 14829, 14975]),
 'interval_widths': array([0.95005001, 0.95006576, 0.95001116, 0.95005001, 0.95024798,
        0.9514941 ])}